In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as et

In [107]:
payments = pd.read_csv('payments.csv', usecols = ['Дата оплаты', 'Валюта', 'Сумма'], parse_dates = ['Дата оплаты'])

In [109]:
payments = payments [payments['Валюта'] !='RUB']

In [111]:
payments

,Дата оплаты,Валюта,Сумма
8,2021-05-07,EUR,187.27
10,2021-12-27,EUR,456.95
13,2021-07-23,USD,788.81
15,2021-05-03,EUR,160.44
17,2021-11-13,USD,583.45
...,...,...,...
987,2021-08-19,EUR,1180.74
990,2021-12-30,EUR,760.29
996,2021-12-07,EUR,2061.54
997,2021-09-30,USD,1657.81


In [113]:
dates = payments['Дата оплаты'].unique()

In [115]:
dates

<DatetimeArray>
['2021-05-07 00:00:00', '2021-12-27 00:00:00', '2021-07-23 00:00:00',
 '2021-05-03 00:00:00', '2021-11-13 00:00:00', '2021-11-19 00:00:00',
 '2021-06-21 00:00:00', '2021-09-02 00:00:00', '2021-02-19 00:00:00',
 '2021-12-06 00:00:00',
 ...
 '2021-06-11 00:00:00', '2021-08-01 00:00:00', '2021-09-06 00:00:00',
 '2021-07-06 00:00:00', '2021-11-09 00:00:00', '2021-03-14 00:00:00',
 '2021-10-23 00:00:00', '2021-08-19 00:00:00', '2021-12-30 00:00:00',
 '2021-09-30 00:00:00']
Length: 186, dtype: datetime64[ns]

In [117]:
payments['Валюта'].unique()

array(['EUR', 'USD'], dtype=object)

In [119]:
tdates = dates[:1]

In [121]:
tdates

<DatetimeArray>
['2021-05-07 00:00:00']
Length: 1, dtype: datetime64[ns]

In [123]:
rates = []
for d in dates:
    dt = pd.to_datetime(d).strftime('%d/%m/%Y')
    url = 'http://cbr.ru/scripts/XML_daily.asp?date_req=' + dt
    r = requests.get(url)
    xml_usd = et.fromstring(r.text)[10]
    usd = float(str.replace(xml_usd[4].text,',', '.'))
    xml_eur = et.fromstring(r.text)[11]
    eur = float(str.replace(xml_eur[4].text,',', '.'))
    dt_rate = [pd.to_datetime(d), usd, eur]
    rates.append(dt_rate)

In [124]:
rates_df = pd.DataFrame(data = rates, columns = ['Дата', 'USD','EUR'])

In [125]:
rates_df

,Дата,USD,EUR
0,2021-05-07,74.5770,89.6788
1,2021-12-27,73.1886,82.9300
2,2021-07-23,73.6945,86.9227
3,2021-05-03,74.8451,90.5850
4,2021-11-13,71.8118,82.2173
...,...,...,...
181,2021-03-14,73.5081,87.7981
182,2021-10-23,70.8623,82.4979
183,2021-08-19,73.4633,86.1210
184,2021-12-30,73.6514,83.2040


In [49]:
rates_df['RUB'] = 1.0

In [51]:
rates_df.to_csv('my_rates.csv', index = False)

In [129]:
payments = pd.read_csv('payments.csv', usecols = ['ФИО', 'Дата рождения'], parse_dates = ['Дата рождения'])

In [131]:
users = payments.drop_duplicates()

In [133]:
users[users.duplicated(subset = ['ФИО'], keep = False)]

,ФИО,Дата рождения
5,Беляева Вероника Александровна,1965-12-09
124,Беляева Вероника Александровна,1994-03-08


In [135]:
users['Последняя буква'] = users['ФИО'].str[-1]

C:\Users\Тимур\AppData\Local\Temp\ipykernel_15780\500864007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['Последняя буква'] = users['ФИО'].str[-1]


In [137]:
users['Последняя буква'].unique()

array(['ч', 'а'], dtype=object)

In [139]:
import numpy as np

In [19]:
users['Пол'] = np.where(users['Последняя буква'] == 'ч', 'Мужчины', 'Женщины')

C:\Users\Тимур\AppData\Local\Temp\ipykernel_15780\3863356004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['Пол'] = np.where(users['Последняя буква'] == 'ч', 'Мужчины', 'Женщины')


In [141]:
users

,ФИО,Дата рождения,Последняя буква
0,Матвеев Виктор Васильевич,1975-12-12,ч
1,Федорова Александра Алексеевна,1990-05-02,а
2,Лавров Данила Тимофеевич,1967-07-06,ч
3,Волошин Даниил Дмитриевич,1964-10-01,ч
4,Харитонова Серафима Глебовна,1975-01-31,а
...,...,...,...
988,Кудрявцева София Григорьевна,1960-11-04,а
989,Тихомирова Анастасия Александровна,1967-03-02,а
992,Лапшина София Владимировна,2001-01-03,а
998,Верещагина Кристина Платоновна,1959-06-16,а


In [143]:
users.drop_duplicates(subset = ['ФИО'], inplace = True)

C:\Users\Тимур\AppData\Local\Temp\ipykernel_15780\4121569145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users.drop_duplicates(subset = ['ФИО'], inplace = True)


In [25]:
users

,ФИО,Дата рождения,Последняя буква,Пол
0,Матвеев Виктор Васильевич,1975-12-12,ч,Мужчины
1,Федорова Александра Алексеевна,1990-05-02,а,Женщины
2,Лавров Данила Тимофеевич,1967-07-06,ч,Мужчины
3,Волошин Даниил Дмитриевич,1964-10-01,ч,Мужчины
4,Харитонова Серафима Глебовна,1975-01-31,а,Женщины
...,...,...,...,...
988,Кудрявцева София Григорьевна,1960-11-04,а,Женщины
989,Тихомирова Анастасия Александровна,1967-03-02,а,Женщины
992,Лапшина София Владимировна,2001-01-03,а,Женщины
998,Верещагина Кристина Платоновна,1959-06-16,а,Женщины


In [147]:
!pip install dadata

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from dadata import Dadata
token = "28e3a371d8d97afff3871cd403a528cdde59e5e6"
dadata = Dadata(token)

In [11]:
payments = pd.read_csv('payments.csv', usecols = ['Адрес доставки'])

In [13]:
payments.drop_duplicates(inplace = True)

In [15]:
len(payments)

855

In [17]:
payments.reset_index(inplace = True, drop = True)

In [19]:
payments

,Адрес доставки
0,"199106, Санкт-Петербург г, Средний В.О. пр-кт,..."
1,"191025, Санкт-Петербург г, Дмитровский пер, до..."
2,"115408, Москва г, Алма-Атинская ул, дом 4"
3,"195197, Санкт-Петербург г, Кондратьевский пр-к..."
4,"125008, Москва г, Черепановых проезд, дом 38,..."
...,...
850,"107150, Москва г, Ивантеевская ул, дом 1, кор..."
851,"127562, Москва г, Декабристов ул, дом 2, корпус"
852,"142520, Московская обл, Павлово-Посадский р-н,..."
853,"141604, Московская обл, Клинский р-н, Клин г, ..."


In [21]:
points = []
for rowIndex, row in payments.iterrows():
    address = payments.iloc[rowIndex]['Адрес доставки']
    result = dadata.suggest('address', address)
    r = result[0]['data']
    lat = r['geo_lat']
    lon = r['geo_lon']
    city = r['city']
    point = [address, city, lat, lon]
    points.append(point)

IndexError: list index out of range

In [23]:
geo_df = pd.DataFrame(data = points, columns = ['Адрес', 'Город', 'Широта', 'Долгота'])

In [25]:
geo_df

,Адрес,Город,Широта,Долгота
0,"199106, Санкт-Петербург г, Средний В.О. пр-кт,...",Санкт-Петербург,59.931995,30.242619
1,"191025, Санкт-Петербург г, Дмитровский пер, до...",Санкт-Петербург,59.929745,30.349762
2,"115408, Москва г, Алма-Атинская ул, дом 4",Москва,55.6413,37.767582
3,"195197, Санкт-Петербург г, Кондратьевский пр-к...",Санкт-Петербург,59.97056,30.382316
4,"125008, Москва г, Черепановых проезд, дом 38,...",Москва,55.841584,37.530858
5,"197350, Санкт-Петербург г, Королева пр-кт, дом...",Санкт-Петербург,60.032461,30.243671
6,"127018, Москва г, Ямская 1-я ул, дом 3/7",Москва,55.794361,37.609344
7,"123154, Москва г, Маршала Тухачевского ул, дом...",Москва,55.784092,37.476635
8,"198326, Санкт-Петербург г, Политрука Пасечника...",Санкт-Петербург,59.779762,30.106947
9,"109147, Москва г, Нижегородская ул, дом 2, ко...",Москва,55.73846,37.674499


In [27]:
geo_df.to_csv('my_geo.csv', index = False)